# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('crewPredict').getOrCreate()

In [3]:
raw_data = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

In [4]:
raw_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
from pyspark.ml.feature import StringIndexer

In [6]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_index')

In [7]:
indexed_raw_data = indexer.fit(raw_data).transform(raw_data)

In [8]:
indexed_raw_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|       

In [11]:
indexed_raw_data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index']

In [9]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [16]:
assembler = VectorAssembler(inputCols=['Cruise_line_index','Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density'], 
                           outputCol='features')

In [17]:
output = assembler.transform(indexed_raw_data)

In [18]:
output.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index',
 'features']

In [19]:
transformed_input_data = output.select('features', 'crew')

In [20]:
train_data, test_data = transformed_input_data.randomSplit([0.7, 0.3])

In [21]:
train_data.show()

+--------------------+-----+
|            features| crew|
+--------------------+-----+
|[0.0,5.0,160.0,36...| 13.6|
|[0.0,9.0,90.09,25...| 8.69|
|[0.0,10.0,90.09,2...| 8.58|
|[0.0,10.0,138.0,3...|11.85|
|[0.0,12.0,90.09,2...| 8.68|
|[0.0,15.0,78.491,...|  6.6|
|[0.0,16.0,74.137,...|  7.6|
|[0.0,17.0,70.0,20...|  7.2|
|[0.0,18.0,70.0,18...|  7.2|
|[0.0,21.0,73.941,...| 8.22|
|[0.0,22.0,73.941,...| 8.22|
|[0.0,23.0,48.563,...| 6.71|
|[1.0,6.0,110.2389...| 11.5|
|[1.0,8.0,110.0,29...| 11.6|
|[1.0,9.0,88.5,21....| 10.3|
|[1.0,10.0,110.0,2...| 11.6|
|[1.0,11.0,110.0,2...| 19.1|
|[1.0,12.0,88.5,21...|10.29|
|[1.0,15.0,70.367,...|  9.2|
|[1.0,15.0,70.367,...|  9.2|
+--------------------+-----+
only showing top 20 rows



In [23]:
from pyspark.ml.regression import LinearRegression

In [24]:
lr = LinearRegression(featuresCol='features', labelCol='crew')

In [25]:
lr_model = lr.fit(train_data)

In [26]:
test_results = lr_model.evaluate(test_data)

In [27]:
test_results.rootMeanSquaredError

0.6946652284011834

In [28]:
test_results.r2

0.9659847363266186

In [29]:
unlabeled_data = test_data.select('features')

In [30]:
prediction = lr_model.transform(unlabeled_data)

In [31]:
prediction.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[0.0,4.0,220.0,54...|21.687022986957476|
|[0.0,6.0,158.0,43...|14.294904376365672|
|[0.0,7.0,158.0,43...| 14.25390217231206|
|[0.0,11.0,90.09,2...| 8.952384195653735|
|[0.0,11.0,138.0,3...|13.369779107086135|
|[0.0,12.0,138.0,3...|13.369760619761344|
|[0.0,13.0,138.0,3...|13.369742132436556|
|[0.0,14.0,138.0,3...|13.369723645111765|
|[0.0,16.0,78.491,...| 8.279370550450945|
|[0.0,17.0,74.137,...| 8.702591790084632|
|[0.0,25.0,73.192,...| 8.661630480043073|
|[1.0,9.0,110.0,29...|12.337537295498123|
|[1.0,11.0,86.0,21...| 9.575681082020093|
|[1.0,12.0,88.5,21...|10.473223252639364|
|[1.0,13.0,101.509...|11.400705954747345|
|[1.0,14.0,101.509...|10.904550494607644|
|[2.0,8.0,77.499,1...| 8.641793597227952|
|[2.0,9.0,116.0,26...|11.412311530770946|
|[2.0,10.0,91.6270...|  9.28956038212386|
|[2.0,11.0,108.977...|11.301727260387178|
+--------------------+------------